In [384]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [385]:
torch.manual_seed(1)
n_classes = 5
y = torch.randint(0, n_classes, (4,))
window_size = 4
# y = F.one_hot(y, num_classes=10)

x = torch.rand(window_size,2)
w = torch.rand(2,n_classes,requires_grad=True)
x_1 = x @ w

# Softmax
# x = F.softmax(x,dim=1)
x_1_exp =  x_1.exp()
probs = x_1_exp/x_1_exp.sum(dim=1).unsqueeze(1)
loss = -probs[torch.arange(window_size), y].log().mean()
print(loss)
loss.backward()

for t in [probs,w]:
  t.retain_grad()

tensor(1.7352, grad_fn=<NegBackward0>)


In [386]:
w_gradient = torch.clone(w._grad)
w_gradient

tensor([[ 0.0814,  0.0924,  0.0859, -0.0889, -0.1708],
        [-0.0688,  0.1538,  0.1344, -0.0244, -0.1950]])

### Gradient Descent

In [387]:
lr = 0.01
w = w -  lr * w_gradient

### Gradient Descent with Momentum

In [388]:
v = torch.zeros_like(w).float()
lr = 0.01
beta = 0.99

v = beta* v + (1.0-beta)  * w_gradient
w = w - lr * v

### Adagrad

In [389]:
v = torch.zeros_like(w).float()
lr = 0.01
epsilon = 0.99

v = v  + w_gradient**2
w = w - (lr/(v**0.5+epsilon ))* v

### RMSProp - Root Mean Square Propagation

In [390]:
v = torch.zeros_like(w).float()

lr = 0.01
epsilon = 0.99
beta = 0.99

v = beta * v  + ( 1- beta ) * w_gradient**2
w = w - (lr/(v**0.5+epsilon ))* w_gradient

### Adam

In [391]:
beta_1 = 0.99
beta_2 = 0.99
step_no = 1.
epsilon = 0.99

m = torch.zeros_like(w).float()
v = torch.zeros_like(w).float()

m = beta_1 * m + ( 1 - beta_1 ) * w_gradient
v = beta_2 * v + ( 1 - beta_2 ) * w_gradient ** 2

m_hat = m / (1 - beta_1**step_no)
v_hat = v / (1 - beta_2**step_no)

w = w - (lr*m_hat/v_hat**0.5+epsilon)*w_gradient